# Comparaison between Python and Matlab for deconvolution (and denoising)

## Google Colab Integration

In [1]:
# !git clone https://github.com/Stage-SuperResolution/NAME-REPO.git
# !mv NAME-REPO/* .
# !rm -rf NAME-REPO
# !pip install git+https://github.com/akhaten/lasp.git@COMPLETE-COMMIT

## Import

In [2]:
# Lasp module

import sys
sys.path.append('../../..')

import lasp.io
import lasp.filters.linear
import lasp.noise
import lasp.convert
import lasp.algorithm.experimental
import lasp.thresholding
import lasp.differential
import lasp.utils

# Other

import scipy.signal
import scipy.io.matlab
import numpy
import pandas
import tqdm
import pathlib
import matplotlib.pyplot

IMAGE_PATH = pathlib.Path('../0-Images')
PYMAT_PATH = pathlib.Path('./1-PythonVsMatlab')
if not(PYMAT_PATH.exists()):
    PYMAT_PATH.mkdir()

## Test On

In [3]:

def mumford_shah_deconv_v1(
    y: numpy.ndarray, 
    h: numpy.ndarray, 
    alpha: float,
    beta: float,
    sigma: float,
    nb_iterations: int,
    tolerance: float,
    error_history: list[float] = None
) -> numpy.ndarray:

    """Mumford Shah
    
    Solve argmin_{x} { (alpha/2) || y - Hx ||^2 + (beta/2) || nabla y ||^2 + || nabla y ||_1 }
    """

    Dx = lasp.differential.dx
    Dy = lasp.differential.dy
    Dxt = lasp.differential.dxT
    Dyt = lasp.differential.dyT

    # Build kernel
    uker = numpy.zeros_like(y)

    laplacian = lasp.filters.linear.laplacian()
    lap_diag = lasp.utils.fourier_diagonalization(
        kernel = laplacian,
        shape_out = y.shape 
    )
   
    h_diag = lasp.utils.fourier_diagonalization(
        kernel = h,
        shape_out = y.shape
    )
    

    h2_diag = numpy.abs(h_diag)**2


    uker = alpha * h2_diag + (beta+sigma) * lap_diag

    rhs1fft = alpha * numpy.conj(h_diag) * numpy.fft.fft2(y)

    # Initialization
    u = numpy.copy(y) 
    d_x=numpy.zeros_like(y)
    d_y=numpy.zeros_like(y)
    b_x=numpy.zeros_like(y)
    b_y=numpy.zeros_like(y)

    for _ in range(0, nb_iterations):

        rhs2 = sigma*Dxt(d_x-b_x)+sigma*Dyt(d_y-b_y)
        rhsfft = rhs1fft + numpy.fft.fft2(rhs2)

        u0=numpy.copy(u)
        
        u = numpy.real(numpy.fft.ifft2(rhsfft / uker))    

        err = numpy.linalg.norm(u-u0, 'fro') / numpy.linalg.norm(u, 'fro')
        
        if not(error_history is None):
            error_history.append(err)

        if err < tolerance:
            break
        
        u_dx, u_dy = Dx(u), Dy(u)

        d_x, d_y = lasp.thresholding.multidimensional_soft(
            d = numpy.array([ u_dx + b_x, u_dy + b_y ]),
            epsilon = 1/sigma
        )

        b_x += (u_dx - d_x)
        b_y += (u_dy - d_y)

    u_normalized = lasp.utils.normalize(u)

    return u_normalized

## Dataset utils

In [4]:
def image_from(params: pandas.Series) -> numpy.ndarray:

    image_path = params['image']
    blur = params['blur']
    noise = params['noise']
    
    img = lasp.io.read(image_path).astype(numpy.double)
    out = numpy.copy(img)

    if pandas.notna(blur):
        kernel = lasp.filters.linear.gaussian_filter(size=blur[0], sigma=blur[1])
        out = scipy.signal.convolve2d(out, kernel, mode='same')
    
    if pandas.notna(noise):
        out = lasp.noise.awgn(out, snr=noise)

    return out

def add(
    dataset: pandas.DataFrame,
    image: pathlib.Path,
    deblur_kernel: tuple[int, float],
    alpha: float,
    beta: float,
    sigma: float,
    tol: float = 10**(-4),
    iterations: int = 300,
    noise: float = numpy.nan,
    blur: tuple[int, float] = numpy.nan
) -> pandas.DataFrame:

    to_add = pandas.DataFrame(
        {
            'image' : [image], 
            'deblur_kernel' : [deblur_kernel], 
            'alpha' : [alpha], 
            'beta' : [beta],
            'sigma' : [sigma],
            'tol' : [tol], 
            'iterations' : [iterations], 
            'noise' : [noise],
            'blur' : [blur]
        }
    )

    return pandas.concat([dataset, to_add], ignore_index=True)
"""
def process_from_index(dataset: pandas.DataFrame, index: int) -> tuple[numpy.ndarray, numpy.ndarray]:
   

    params = dataset.loc[index] # Get params from dataset


    out = image_from(params)
    normalized = lasp.utils.normalize(out)

    deblur_kernel = lasp.filters.linear.gaussian_filter(
        size = dataset.iloc[index]['deblur_kernel'][0],
        sigma = dataset.iloc[index]['deblur_kernel'][1]
    )

    # Hyper-parameters
    alpha = params['alpha']
    beta = params['beta']
    sigma = params['sigma']

    # Iterative parameters
    tol = params['tol']
    iterations = params['iterations']

    errors = []

    res = mumford_shah_deconv_v1(
        normalized, deblur_kernel,
        alpha, beta, sigma,
        iterations, tol, 
        errors
    )

    return res, errors
"""

"\ndef process_from_index(dataset: pandas.DataFrame, index: int) -> tuple[numpy.ndarray, numpy.ndarray]:\n   \n\n    params = dataset.loc[index] # Get params from dataset\n\n\n    out = image_from(params)\n    normalized = lasp.utils.normalize(out)\n\n    deblur_kernel = lasp.filters.linear.gaussian_filter(\n        size = dataset.iloc[index]['deblur_kernel'][0],\n        sigma = dataset.iloc[index]['deblur_kernel'][1]\n    )\n\n    # Hyper-parameters\n    alpha = params['alpha']\n    beta = params['beta']\n    sigma = params['sigma']\n\n    # Iterative parameters\n    tol = params['tol']\n    iterations = params['iterations']\n\n    errors = []\n\n    res = mumford_shah_deconv_v1(\n        normalized, deblur_kernel,\n        alpha, beta, sigma,\n        iterations, tol, \n        errors\n    )\n\n    return res, errors\n"

In [6]:
def make_dataset() -> pandas.DataFrame:    
    

    dataset = pandas.DataFrame(
        columns = [
            'image' , 'deblur_kernel',
            'alpha', 'beta', 'sigma', # Hyper-parameters
            'tol', 'iterations', # Iterative parameters
            'noise',
            'blur'
        ]
    )

    dataset = add(
        dataset = dataset, 
        image = IMAGE_PATH / 'Baboon.bmp', deblur_kernel = (7, 3),
        alpha = 100., beta = 1., sigma = 2., 
        tol = 1e-4, iterations = 300, 
        noise = lasp.convert.snrdb_to_snr(30),
        blur = (7, 3)
    )

    return dataset

In [7]:
DATASET_PATH = PYMAT_PATH / pathlib.Path('dataset.pkl')
dataset = make_dataset()
dataset

,image,deblur_kernel,alpha,beta,sigma,tol,iterations,noise,blur
0,..\0-Images\Baboon.bmp,"(7, 3)",100.0,1.0,2.0,0.0001,300,31.622777,"(7, 3)"


In [8]:
pandas.to_pickle(dataset, DATASET_PATH)

In [9]:
""""
MAT_PATH = PYMAT_PATH / 'datas.mat'

def make_mat(dataset: pandas.DataFrame, index: int) -> dict:
    datas = dataset.loc[index].to_dict()
    img = image_from(dataset.loc[index])
    normalized = lasp.utils.normalize(img)
    #print(datas.keys())
    datas['image'] = normalized
    datas['deblur_kernel'] = lasp.filters.linear.gaussian_filter(
        size=datas['deblur_kernel'][0],
        sigma=datas['deblur_kernel'][1]
    )
    #print(datas['deblur_kernel'])
    return datas

matlab = make_mat(dataset, index=0)
scipy.io.matlab.savemat(MAT_PATH, matlab)
pc_fixe = pathlib.Path('../../../../Seg_ConVar_M-S_Thd_SIIMS_1/data/datas.mat')
if pc_fixe.exists():
    scipy.io.matlab.savemat(pc_fixe, matlab)
"""

'"\nMAT_PATH = PYMAT_PATH / \'datas.mat\'\n\ndef make_mat(dataset: pandas.DataFrame, index: int) -> dict:\n    datas = dataset.loc[index].to_dict()\n    img = image_from(dataset.loc[index])\n    normalized = lasp.utils.normalize(img)\n    #print(datas.keys())\n    datas[\'image\'] = normalized\n    datas[\'deblur_kernel\'] = lasp.filters.linear.gaussian_filter(\n        size=datas[\'deblur_kernel\'][0],\n        sigma=datas[\'deblur_kernel\'][1]\n    )\n    #print(datas[\'deblur_kernel\'])\n    return datas\n\nmatlab = make_mat(dataset, index=0)\nscipy.io.matlab.savemat(MAT_PATH, matlab)\npc_fixe = pathlib.Path(\'../../../../Seg_ConVar_M-S_Thd_SIIMS_1/data/datas.mat\')\nif pc_fixe.exists():\n    scipy.io.matlab.savemat(pc_fixe, matlab)\n'

## Process Dataset

In [10]:
begin = 0
dataset = pandas.read_pickle(DATASET_PATH)
dataset_filtered = dataset.loc[begin:]

RESULTS_PATH = PYMAT_PATH / pathlib.Path('./results')
if not(RESULTS_PATH.exists()):
    RESULTS_PATH.mkdir()


i = 0

RES_CURRENT_PATH = RESULTS_PATH / str(i)
if not(RES_CURRENT_PATH.exists()):
    RES_CURRENT_PATH.mkdir()


params = dataset_filtered.loc[i] # Get params from dataset

out = image_from(params)
normalized = lasp.utils.normalize(out)

deblur_kernel = lasp.filters.linear.gaussian_filter(
    size = dataset_filtered.loc[i]['deblur_kernel'][0],
    sigma = dataset_filtered.loc[i]['deblur_kernel'][1]
)

# Hyper-parameters
alpha = params['alpha']
beta = params['beta']
sigma = params['sigma']

# Iterative parameters
tol = params['tol']
iterations = params['iterations']

# Matlab datas
matlab = params.to_dict()
matlab['image'] = normalized
matlab['deblur_kernel'] = deblur_kernel
MAT_PATH = PYMAT_PATH / 'datas.mat'
scipy.io.matlab.savemat(MAT_PATH, matlab)
pc_fixe = pathlib.Path('../../../../Seg_ConVar_M-S_Thd_SIIMS_1/data/datas.mat')
if pc_fixe.exists():
    scipy.io.matlab.savemat(pc_fixe, matlab)

# Algorithm
errors = []

res = mumford_shah_deconv_v1(
    normalized, deblur_kernel,
    alpha, beta, sigma,
    iterations, tol, 
    errors
)

lasp.io.save(res, RES_CURRENT_PATH / 'result.npy')
lasp.io.save(res, RES_CURRENT_PATH / 'result.png')
lasp.io.save(errors, RES_CURRENT_PATH / 'errors.npy')

## Comparaison with Matlab results

In [12]:
res_mat = scipy.io.matlab.loadmat(PYMAT_PATH / 'uu.mat')['uu']
res_py = lasp.io.read(RESULTS_PATH / '0' / 'result.npy')
print(numpy.max(numpy.abs(res_py-res_mat)))

2.9976021664879227e-15
